# Repressilator split between 2 strains

This repressilator is split between 2 strains - one stops producing red fluorescent protein and c14 in response to c4. Second strain has yellow fluorescence and tetR synthesis repressed by c14, and tetR in turn represses cyan fluorescence and c4 synthesis.

In [ ]:
pip install -e .

In [ ]:
import loica as lc
import matplotlib.pyplot as plt
import networkx as nx
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def growth_rate(t):
    return lc.gompertz_growth_rate(t, 0.005, 0.5, 1, 1)

def biomass(t):
    return lc.gompertz(t, 0.005, 0.5, 1, 1)

# No AiiA

## Strain 1 (one node)

In [ ]:
gn1 = lc.GeneticNetwork()
metab1 = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
strain_1 = lc.Strain('Strain 1', gn1, metab1)

In [ ]:
# Transcription factors
laci_1 = lc.Regulator(name='LacI', degradation_rate=1, init_concentration=10)

# Signalling molecules
c4_1 = lc.Regulator(name='C4', diffusion_rate=100)
c14_1 = lc.Regulator(name='C14', diffusion_rate=100)

gn1.add_regulator([laci_1, c4_1, c14_1])

In [ ]:
cini = lc.Regulator(name='CinI', degradation_rate=1)
gn1.add_regulator(cini)

In [ ]:
mcherry = lc.Reporter(name='mCherry', degradation_rate=1, color='red') 
gn1.add_reporter(mcherry)

In [ ]:
aiia_1 = lc.Regulator(name='AiiA', degradation_rate=1)
gn1.add_regulator(aiia_1)

j23100_1 = lc.Source(output=aiia_1, rate=1, name='J23100')
gn1.add_operator(j23100_1)

lactonase_1 = lc.Degrader(name='Lactonase', enzyme=aiia_1, substrate=[c4_1, c14_1], Km=10000, k2=2)
gn1.add_impactor(lactonase_1)


In [ ]:
c4_laci = lc.Hill1(name='Prhl', input=c4_1, output=[laci_1,mcherry], alpha=[0, 10000], K=1, n=2)
laci_not_cini = lc.Hill1(name='Plac', input=laci_1, output=cini, alpha=[10000, 0], K=1, n=2)
cini_c14 = lc.Producer(name='CinI', enzyme=cini, product=c14_1, k2=0.01)

gn1.add_operator([c4_laci, laci_not_cini])
gn1.add_impactor(cini_c14)

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=100)
gn1.draw() #pos=nx.planar_layout

## Strain 2 (two nodes)

In [ ]:
gn2 = lc.GeneticNetwork()
metab2 = lc.SimulatedMetabolism("Simulated metabolism", biomass, growth_rate)
strain_2 = lc.Strain('Strain 2', gn2, metab2)

In [ ]:
# Transcription factors
laci_2 = lc.Regulator(name='LacI', degradation_rate=1)
tetr_2 = lc.Regulator(name='TetR', degradation_rate=1)

# Signalling molecules
c4_2 = lc.Regulator(name='C4', diffusion_rate=1)
c14_2 = lc.Regulator(name='C14', diffusion_rate=1)

gn2.add_regulator([tetr_2, laci_2, c4_2, c14_2])

In [ ]:
rhli = lc.Regulator(name='RhlI', degradation_rate=1)
gn2.add_regulator(rhli)

In [ ]:
cyan = lc.Reporter(name='CFP', degradation_rate=1, color='blue') 
mven = lc.Reporter(name='mVenus', degradation_rate=1, color='yellow') 

gn2.add_reporter([cyan, mven])

In [ ]:
aiia_2 = lc.Regulator(name='AiiA', degradation_rate=1)
gn2.add_regulator(aiia_2)

j23100_2 = lc.Source(output=aiia_2, rate=1, name='J23100')
gn2.add_operator(j23100_2)

lactonase_2 = lc.Degrader(name='Lactonase', enzyme=aiia_2, substrate=[c4_2, c14_2], Km=10000, k2=2)
gn2.add_impactor(lactonase_2)

In [ ]:

# One node
c14_laci = lc.Hill1(name='Pcin', input=c14_2, output=[laci_2, mven], alpha=[0, 10000], K=1, n=2)
laci_not_tetr = lc.Hill1(name='Plac', input=laci_2, output=[tetr_2, cyan], alpha=[10000, 0], K=1, n=2)
# Second node
tetr_not_c4 = lc.Hill1(name='Ptet', input=tetr_2, output=rhli, alpha=[10000, 0], K=1, n=2)
rhli_c4 = lc.Producer(name='RhlI', enzyme=rhli, product=c4_2, k2=0.01)

gn2.add_operator([c14_laci, laci_not_tetr, tetr_not_c4])
gn2.add_impactor(rhli_c4)

In [ ]:
plt.figure(figsize=(3.5,3.5), dpi=100)
gn2.draw() #pos=nx.planar_layout

In [ ]:
sample = lc.Sample(strain=[strain_1, strain_2])
# # set starting external concentration
# sample.set_ext_conc('C4', 50)
assay = lc.Assay([sample], 
                n_measurements=100, 
                interval=0.24,
                name='Extended repressilator',
                description='Simulated expression of extended repressilator strain'
                )

In [ ]:
assay.run(substeps=100, mode="track_all")

In [ ]:
m = assay.measurements
fig, ax = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
fig4, ax4 = plt.subplots()
m[m.Signal=='mCherry in Strain 1'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='LacI in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='C4 in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='C14 in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='CinI in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='AiiA in Strain 1'].plot(x='Time', y='Measurement', style='-', ax=ax4)
ax.legend(['mCherry'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')
ax1.legend(['LacI in S1'])
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')
ax2.legend(['C4 in S1', 'C14 in S1'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
ax3.legend(['CinI'])
ax3.set_xlabel('Time (h)')
ax3.set_ylabel('Concentration (M)')
ax4.legend(['AiiA'])
ax4.set_xlabel('Time (h)')
ax4.set_ylabel('Concentration (M)')

In [ ]:
fig, ax = plt.subplots()
fig1, ax1 = plt.subplots()
fig2, ax2 = plt.subplots()
fig3, ax3 = plt.subplots()
fig4, ax4 = plt.subplots()
m[m.Signal=='CFP in Strain 2'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain 2'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
m[m.Signal=='LacI in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='TetR in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax1)
m[m.Signal=='C4 in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='C14 in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='RhlI in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax3)
m[m.Signal=='AiiA in Strain 2'].plot(x='Time', y='Measurement', style='-', ax=ax4)
ax.legend(['CFP', 'mVenus'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')
ax1.legend(['LacI in S2', 'TetR'])
ax1.set_xlabel('Time (h)')
ax1.set_ylabel('Concentration (M)')
ax2.legend(['C4 in S2', 'C14 in S2'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
ax3.legend(['RhlI'])
ax3.set_xlabel('Time (h)')
ax3.set_ylabel('Concentration (M)')
ax4.legend(['AiiA'])
ax4.set_xlabel('Time (h)')
ax4.set_ylabel('Concentration (M)')

In [ ]:
fig, ax = plt.subplots()

m[m.Signal=='mCherry in Strain 1'].plot(x='Time', y='Measurement', color='red', style='-', ax=ax)
m[m.Signal=='CFP in Strain 2'].plot(x='Time', y='Measurement', color='cyan', style='-', ax=ax)
m[m.Signal=='mVenus in Strain 2'].plot(x='Time', y='Measurement', color='yellow', style='-', ax=ax)
ax.legend(['mCherry','CFP', 'mVenus'])
ax.set_xlabel('Time (h)')
ax.set_ylabel('Concentration (M)')



In [ ]:
fig2, ax2 = plt.subplots()

m[m.Signal=='Extracellular C4'].plot(x='Time', y='Measurement', style='-', ax=ax2)
m[m.Signal=='Extracellular C14'].plot(x='Time', y='Measurement', style='-', ax=ax2)

ax2.legend(['Ext C4', 'Ext C14'])
ax2.set_xlabel('Time (h)')
ax2.set_ylabel('Concentration (M)')
